## Controlas Fechas de Actualización de las Tablas

- monitorear_actualizacion_tablas.py
- Objetivo: Verificar que las tablas críticas del esquema `src` estén actualizadas con datos recientes.
- Autor: [Zeetrex]
- Fecha: [Auto-generado]

In [1]:
import psycopg2 as pg2
from datetime import date
import time
import logging
import os
import sys
from dotenv import dotenv_values
import pandas as pd

# Configurar logging
logging.basicConfig(
    filename='./logs/monitoreo_tablas.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

ENV_PATH = os.environ.get("ETL_ENV_PATH", "C:/ETL/ETL_DIARCO/.env")  # Toma Producción si está definido, o la ruta por defecto E:\ETL\ETL_DIARCO\.env
# Verificar si el archivo .env existe
if not os.path.exists(ENV_PATH):
    print(f"El archivo .env no existe en la ruta: {ENV_PATH}")
    print(f"Directorio actual: {os.getcwd()}")
    sys.exit(1)
    
secrets = dotenv_values(ENV_PATH)

def Open_Diarco_Data(): 
    conn_str = f"dbname={secrets['PG_DB']} user={secrets['PG_USER']} password={secrets['PG_PASSWORD']} host={secrets['PG_HOST']} port={secrets['PG_PORT']}"
    #print (conn_str)
    for i in range(5):
        try:    
            conn = pg2.connect(conn_str)
            return conn
        except Exception as e:
            print(f'Error en la conexión: {e}')
            time.sleep(5)
    return None  # Retorna None si todos los intentos fallan


In [ ]:

# Contiene campo fecha_extraccion
tablas_a_controlar = [
    "base_forecast_articulos", "base_productos_vigentes", "base_stock_sucursal", "t020_proveedor",
    "m_3_articulos", "t020_proveedor_gestion_compra", "t050_articulos", "t051_articulos_sucursal", 
    "t020_proveedor", "t060_stock", "t080_oc_cabe", "t081_oc_deta", "t100_empresa_suc","t052_articulos_proveedor",
    "t702_est_vtas_por_articulo_dbarrio", "t710_estadis_oferta_folder", "t710_estadis_precios",
    "t710_estadis_reposicion"
]


In [ ]:

def controlar_tablas(conn):
    cur = conn.cursor()
    for tabla in tablas_a_controlar:
        try:
            query = f"""
                SELECT
                    MAX(fecha_extraccion) AS ultima_fecha,
                    COUNT(*) AS cantidad
                FROM src.{tabla}
                WHERE fecha_extraccion::date = (
                    SELECT MAX(fecha_extraccion::date)
                    FROM src.{tabla})
                )
            """
            
            cur.execute(query)
            resultado = cur.fetchone()
            ultima_fecha, cantidad = resultado

            cur.execute("""
                INSERT INTO src.monitoreo_estado_tablas (tabla, ultima_fecha_extraccion, cantidad_registros)
                VALUES (%s, %s, %s)
                ON CONFLICT (tabla) DO UPDATE SET
                    ultima_fecha_extraccion = EXCLUDED.ultima_fecha_extraccion,
                    cantidad_registros = EXCLUDED.cantidad_registros,
                    fecha_control = NOW()
            """, (tabla, ultima_fecha, cantidad))
            logging.info(f"✔ Tabla {tabla}: {cantidad} registros con fecha {ultima_fecha}")
        except Exception as e:
            logging.error(f"❌ Error en la tabla {tabla}: {str(e)}")
            continue

    conn.commit()
    cur.close()

In [ ]:
import pandas as pd

# Contiene campo fecha_extraccion
tablas_a_controlar = [
    "base_forecast_articulos", "base_productos_vigentes", "base_stock_sucursal", "t020_proveedor",
    "m_3_articulos", "t020_proveedor_gestion_compra", "t050_articulos", "t051_articulos_sucursal", 
    "t020_proveedor", "t060_stock", "t080_oc_cabe", "t081_oc_deta", "t100_empresa_suc","t052_articulos_proveedor",
    "t702_est_vtas_por_articulo_dbarrio", "t710_estadis_oferta_folder", "t710_estadis_precios",
    "t710_estadis_reposicion"
]


columns = ["tabla", "ultima_fecha_extraccion", "cantidad_registros"]
df_control = pd.DataFrame(columns=columns)

conn = Open_Diarco_Data()
cur = conn.cursor()

for tabla in tablas_a_controlar:
    print(f"Procesando tabla: {tabla}")
    try:
        query = f"""
            SELECT
                MAX(fecha_extraccion) AS ultima_fecha,
                COUNT(*) AS cantidad
            FROM src.{tabla}
            WHERE fecha_extraccion::date = (
                SELECT MAX(fecha_extraccion::date)
                FROM src.{tabla})
        """

        print(f"Ejecutando consulta : {query}")
        
        cur.execute(query)
        resultado = cur.fetchone()
        ultima_fecha, cantidad = resultado

        print(f"Tabla: {tabla}, Última fecha: {ultima_fecha}, Cantidad de registros: {cantidad}")

       # Insertar en DataFrame
        nuevo_registro = pd.DataFrame([{
            "tabla": tabla,
            "ultima_fecha_extraccion": ultima_fecha,
            "cantidad_registros": cantidad
        }])

        df_control = pd.concat([df_control, nuevo_registro], ignore_index=True)


    except Exception as e:
        logging.error(f"❌ Error en la tabla {tabla}: {str(e)}")
        continue



In [ ]:
# Tablas a controlar
tablas_originales_a_controlar = [
    "t114_rubros", "t117_compradores", 
    "m_91_sucursales", 
    "m_92_depositos", "m_93_sustitutos", "m_94_alternativos",
    "m_95_sensibles", "m_96_stock_seguridad"    
]

# Diccionario que mapea tabla → campo de fecha
campos_fecha_por_tabla = { 
    "t114_rubros": "f_alta",
    "t117_compradores": "f_modif",
    "m_91_sucursales": "f_proc",
    "m_92_depositos": "f_proc",
    "m_93_sustitutos": "f_proc",
    "m_94_alternativos": "f_proc",
    "m_95_sensibles": "f_proc",
    "m_96_stock_seguridad": "f_proc"    
}


# Loop dinámico con campo de fecha variable
for tabla in tablas_originales_a_controlar:
    campo_fecha = campos_fecha_por_tabla.get(tabla, "fecha_extraccion")  # fallback opcional

    try:
        query = f"""
            SELECT
                MAX({campo_fecha}) AS ultima_fecha,
                COUNT(*) AS cantidad
            FROM src.{tabla}
            WHERE {campo_fecha}::date = (
                SELECT MAX({campo_fecha}::date)
                FROM src.{tabla}
            )
        """

        print(f"Ejecutando consulta : {query}")
        cur.execute(query)
        resultado = cur.fetchone()
        ultima_fecha, cantidad = resultado

        print(f"Tabla: {tabla} | Fecha: {ultima_fecha} | Registros: {cantidad}")

    except Exception as e:
        print(f"Error en la tabla {tabla}: {e}")


       # Insertar en DataFrame
        nuevo_registro = pd.DataFrame([{
            "tabla": tabla,
            "ultima_fecha_extraccion": ultima_fecha,
            "cantidad_registros": cantidad
        }])

        df_control = pd.concat([df_control, nuevo_registro], ignore_index=True)


    except Exception as e:
        logging.error(f"❌ Error en la tabla {tabla}: {str(e)}")
        continue



conn.commit()
cur.close()

In [ ]:

if __name__ == "__main__":
    try:
        conn = Open_Diarco_Data()
        controlar_tablas(conn)
        conn.close()
        logging.info("✅ Monitoreo completado correctamente.")
    except Exception as e:
        logging.critical(f"🚨 Fallo de conexión o ejecución: {str(e)}")


In [ ]:
import pandas as pd
import logging
import time
from typing import List, Dict
from psycopg2 import sql
from psycopg2.extras import DictCursor

def obtener_control_interfaces(tablas: List[str],
                               max_retries: int = 3,
                               retry_delay: float = 2.0) -> pd.DataFrame:
    """
    Devuelve un DataFrame con las columnas:
    - tabla
    - ultima_fecha_extraccion
    - cantidad_registros

    Cada fila corresponde a una tabla de `tablas`.
    """
    registros: List[Dict] = []
    conn = Open_Diarco_Data()
    conn.autocommit = True
    with conn.cursor(cursor_factory=DictCursor) as cur:
        # Opcional: desactivar timeout para consultas complejas
        cur.execute("SET statement_timeout = 0;")
        for tabla in tablas:
            logging.info(f"Procesando tabla: {tabla}")
            consulta = sql.SQL(
                """
                SELECT
                    MAX(fecha_extraccion)      AS ultima_fecha,
                    COUNT(*)                   AS cantidad
                FROM src.{tabla}
                WHERE fecha_extraccion::date = (
                    SELECT MAX(fecha_extraccion::date)
                    FROM src.{tabla}
                );
                """
            ).format(tabla=sql.Identifier(tabla))
            for intento in range(1, max_retries + 1):
                try:
                    cur.execute(consulta)
                    fila = cur.fetchone()
                    registros.append({
                        "tabla": tabla,
                        "ultima_fecha_extraccion": fila["ultima_fecha"],
                        "cantidad_registros": fila["cantidad"]
                    })
                    break
                except Exception as e:
                    logging.error(f"Error en tabla {tabla}, intento {intento}: {e}",
                                  exc_info=True)
                    if intento < max_retries:
                        time.sleep(retry_delay)
                    else:
                        registros.append({
                            "tabla": tabla,
                            "ultima_fecha_extraccion": None,
                            "cantidad_registros": None
                        })
    conn.close()
    df_control = pd.DataFrame(registros,
                              columns=["tabla", "ultima_fecha_extraccion", "cantidad_registros"])
    return df_control

# Uso:
tablas = [
    "base_forecast_articulos", "base_productos_vigentes", "base_stock_sucursal",
    "t020_proveedor", "m_3_articulos", "t020_proveedor_gestion_compra",
    "t050_articulos", "t051_articulos_sucursal", "t060_stock",
    "t080_oc_cabe", "t081_oc_deta", "t100_empresa_suc",
    "t052_articulos_proveedor", "t702_est_vtas_por_articulo_dbarrio",
    "t710_estadis_oferta_folder", "t710_estadis_precios",
    "t710_estadis_reposicion"
]
df_resultado = obtener_control_interfaces(tablas)
print(df_resultado)

In [2]:
import pandas as pd
import logging
import time
from typing import List, Dict
from psycopg2 import sql
from psycopg2.extras import DictCursor

def obtener_control_interfaces(tablas_por_fecha: Dict[str, str],
                               max_retries: int = 3,
                               retry_delay: float = 2.0) -> pd.DataFrame:
    """
    Realiza control de interfaces sobre múltiples tablas con diferentes campos de fecha.
    Retorna un DataFrame con columnas: tabla, campo_fecha, ultima_fecha_extraccion, cantidad_registros.
    """
    registros = []
    conn = Open_Diarco_Data()
    conn.autocommit = True

    with conn.cursor(cursor_factory=DictCursor) as cur:
        cur.execute("SET statement_timeout = 0;")

        for tabla, campo_fecha in tablas_por_fecha.items():
            logging.info(f"Procesando tabla: {tabla} (campo: {campo_fecha})")

            query = sql.SQL("""
                SELECT
                    MAX({campo_fecha}) AS ultima_fecha,
                    COUNT(*) AS cantidad
                FROM src.{tabla}
                WHERE {campo_fecha}::date = (
                    SELECT MAX({campo_fecha}::date)
                    FROM src.{tabla}
                )
            """).format(
                tabla=sql.Identifier(tabla),
                campo_fecha=sql.Identifier(campo_fecha)
            )

            for intento in range(1, max_retries + 1):
                try:
                    cur.execute(query)
                    fila = cur.fetchone()
                    registros.append({
                        "tabla": tabla,
                        "campo_fecha": campo_fecha,
                        "ultima_fecha_extraccion": fila["ultima_fecha"],
                        "cantidad_registros": fila["cantidad"]
                    })
                    break
                except Exception as e:
                    logging.error(f"Error en tabla {tabla}, intento {intento}: {e}", exc_info=True)
                    if intento < max_retries:
                        time.sleep(retry_delay)
                    else:
                        registros.append({
                            "tabla": tabla,
                            "campo_fecha": campo_fecha,
                            "ultima_fecha_extraccion": None,
                            "cantidad_registros": None
                        })

    conn.close()
    return pd.DataFrame(registros,
                        columns=["tabla", "campo_fecha", "ultima_fecha_extraccion", "cantidad_registros"])


In [5]:
# 1. Tablas con campo 'fecha_extraccion'
tablas_fechas_estandar = [
    "base_forecast_articulos", "base_productos_vigentes", "base_stock_sucursal", "t020_proveedor",
    "m_3_articulos", "t020_proveedor_gestion_compra", "t050_articulos", "t051_articulos_sucursal", 
    "t060_stock", "t080_oc_cabe", "t081_oc_deta", "t100_empresa_suc", "t052_articulos_proveedor",
    "t710_estadis_oferta_folder", "t710_estadis_precios",
    "t710_estadis_reposicion"
]
tablas_dict_1 = {tabla: "fecha_extraccion" for tabla in tablas_fechas_estandar}

# 2. Tablas con campo personalizado
tablas_dict_2 = {
    "t114_rubros": "f_alta",
    "t117_compradores": "f_modif",
    "m_91_sucursales": "f_proc",
    "m_92_depositos": "f_proc",
    "m_93_sustitutos": "f_proc",
    "m_94_alternativos": "f_proc",
    "m_95_sensibles": "f_proc",
    "m_96_stock_seguridad": "f_proc",
    "t702_est_vtas_por_articulo" : "f_venta", 
    "t702_est_vtas_por_articulo_dbarrio": "f_venta"
}

# Unificar en un solo diccionario
tablas_total = {**tablas_dict_1, **tablas_dict_2}

# Ejecutar control
df_control_total = obtener_control_interfaces(tablas_total)
print(df_control_total)


                                 tabla       campo_fecha  \
0              base_forecast_articulos  fecha_extraccion   
1              base_productos_vigentes  fecha_extraccion   
2                  base_stock_sucursal  fecha_extraccion   
3                       t020_proveedor  fecha_extraccion   
4                        m_3_articulos  fecha_extraccion   
5        t020_proveedor_gestion_compra  fecha_extraccion   
6                       t050_articulos  fecha_extraccion   
7              t051_articulos_sucursal  fecha_extraccion   
8                           t060_stock  fecha_extraccion   
9                         t080_oc_cabe  fecha_extraccion   
10                        t081_oc_deta  fecha_extraccion   
11                    t100_empresa_suc  fecha_extraccion   
12            t052_articulos_proveedor  fecha_extraccion   
13          t710_estadis_oferta_folder  fecha_extraccion   
14                t710_estadis_precios  fecha_extraccion   
15             t710_estadis_reposicion  